<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/7_nlp_sentiment_transformer/GC7_2_torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.2 torchtextでのDataset、DataLoaderの実装方法

- 本ファイルでは、torchtextを使用してDatasetおよびDataLoaderを実装する方法を解説します。


※ 本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。  
※ This document is customized only for google colaboratory as of 2021/08.

# 7.2 学習目標

1.	torchtextを用いてDatasetおよびDataLoaderの実装ができる

---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

In [ ]:
!ls

In [ ]:
%cd "pytorch_advanced"

In [ ]:
!ls

In [ ]:
%cd "7_nlp_sentiment_transformer"

In [ ]:
!ls

---
# Preparation for word separation (GC7-1)

This is the whole procedure of preparation of word separation.

In [ ]:
# janome, mecab, ipadic-neologd, and mecab-pytorch

!pip install janome > /dev/null

# https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1

%cd ~
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!test -d mecab-ipadic-neologd || git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
!echo yes | ./mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

!test -f /usr/local/etc/mecabrc || ln -s /etc/mecabrc /usr/local/etc/mecabrc 

!echo "Dictionary path is : "
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

%cd -

---
# 7.2 節


- We use torchtext as torchtext.legacy. ... see [issue 148](https://github.com/YutaroOgawa/pytorch_advanced/issues/148)





---
# 1 . 前処理と単語分割の関数を実装


In [ ]:
# 単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer

j_t = Tokenizer()


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


In [ ]:
# 前処理として正規化をする関数を定義
import re


def preprocessing_text(text):
    # 半角・全角の統一
    # 今回は無視

    # 英語の小文字化
    # 今回はここでは無視
    # output = output.lower()

    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    # 記号と数字の除去
    # 今回は無視。半角記号,数字,英字
    # 今回は無視。全角記号

    # 特定文字を正規表現で置換する
    # 今回は無視

    return text


In [ ]:
# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_janome(text)  # Janomeの単語分割

    return ret


# 動作確認
text = "昨日は とても暑く、気温が36度もあった。"
print(tokenizer_with_preprocessing(text))


# 2. 文章データの読み込み

In [ ]:
import torchtext.legacy

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリー（単語集：後で解説）に追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を先頭に用意するかどうか
# fix_length：全部の文章を指定した長さと同じになるように、paddingします


In [ ]:
# data.TabularDataset 詳細
# https://torchtext.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits
%cd '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer'

# フォルダ「data」から各tsvファイルを読み込み、Datasetにします
# 1行がTEXTとLABELで区切られていることをfieldsで指示します
train_ds, val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


# 動作確認
print('訓練データの数', len(train_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))


# 単語の数値化

In [ ]:
# ボキャブラリーを作成します
# 訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー（単語集）を構築
TEXT.build_vocab(train_ds, min_freq=1)

# 訓練データ内の単語と頻度を出力(頻度min_freqより大きいものが出力されます)
TEXT.vocab.freqs  # 出力させる


In [ ]:
# ボキャブラリーの単語をidに変換した結果を出力。
# 頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi  # 出力。string to identifiers 文字列をidへ


# DataLoaderの作成

In [ ]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.legacy.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.legacy.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.legacy.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/03. 